In [35]:
import sagemaker
import json
import boto3

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
prefix = 'sagemaker-studio-book/chapter07'

In [204]:
import numpy as np
import pandas as pd

In [205]:
df=pd.read_csv('s3://sagemaker-sample-files/datasets/tabular/synthetic/churn.txt')
df['CustomerID']=df.index
df.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?,CustomerID
0,PA,163,806,403-2562,no,yes,300,8.162204,3,7.579174,...,6.508639,4.065759,100,5.111624,4.928160,6,5.673203,3,True.,0
1,SC,15,836,158-8416,yes,no,0,10.018993,4,4.226289,...,9.972592,7.141040,200,6.436188,3.221748,6,2.559749,8,False.,1
2,MO,131,777,896-6253,no,yes,300,4.708490,3,4.768160,...,4.566715,5.363235,100,5.142451,7.139023,2,6.254157,4,False.,2
3,WY,75,878,817-5729,yes,yes,700,1.268734,3,2.567642,...,2.333624,3.773586,450,3.814413,2.245779,6,1.080692,6,False.,3
4,WY,146,878,450-4942,yes,no,0,2.696177,3,5.908916,...,3.670408,3.751673,250,2.796812,6.905545,4,7.134343,6,True.,4


In [206]:
df.columns

Index(['State', 'Account Length', 'Area Code', 'Phone', 'Int'l Plan',
       'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge',
       'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
       'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
       'CustServ Calls', 'Churn?', 'CustomerID'],
      dtype='object')

In [207]:
df[["Int'l Plan", "VMail Plan"]] = df[["Int'l Plan", "VMail Plan"]].replace(to_replace=['yes', 'no'], value=[1, 0])

In [210]:
df['Churn?'] = df['Churn?'].replace(to_replace=['True.', 'False.'], value=[1, 0])

In [212]:
df.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?,CustomerID
0,PA,163,806,403-2562,0,1,300,8.162204,3,7.579174,...,6.508639,4.065759,100,5.111624,4.928160,6,5.673203,3,1,0
1,SC,15,836,158-8416,1,0,0,10.018993,4,4.226289,...,9.972592,7.141040,200,6.436188,3.221748,6,2.559749,8,0,1
2,MO,131,777,896-6253,0,1,300,4.708490,3,4.768160,...,4.566715,5.363235,100,5.142451,7.139023,2,6.254157,4,0,2
3,WY,75,878,817-5729,1,1,700,1.268734,3,2.567642,...,2.333624,3.773586,450,3.814413,2.245779,6,1.080692,6,0,3
4,WY,146,878,450-4942,1,0,0,2.696177,3,5.908916,...,3.670408,3.751673,250,2.796812,6.905545,4,7.134343,6,1,4


In [213]:
columns=['Churn?', 'State', 'Account Length', "Int'l Plan",
           'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge',
           'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
           'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
           'CustServ Calls']
df.index = df['CustomerID']
df_final = df[columns]

In [214]:
df_final.head()

,Churn?,State,Account Length,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls
CustomerID,,,,,,,,,,,,,,,,,,,
0,1,PA,163,0,1,300,8.162204,3,7.579174,3.933035,4,6.508639,4.065759,100,5.111624,4.928160,6,5.673203,3
1,0,SC,15,1,0,0,10.018993,4,4.226289,2.325005,0,9.972592,7.141040,200,6.436188,3.221748,6,2.559749,8
2,0,MO,131,0,1,300,4.708490,3,4.768160,4.537466,3,4.566715,5.363235,100,5.142451,7.139023,2,6.254157,4
3,0,WY,75,1,1,700,1.268734,3,2.567642,2.528748,5,2.333624,3.773586,450,3.814413,2.245779,6,1.080692,6
4,1,WY,146,1,0,0,2.696177,3,5.908916,6.015337,3,3.670408,3.751673,250,2.796812,6.905545,4,7.134343,6


In [216]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_final, test_size=0.1, random_state=42, 
                                     shuffle=True, stratify=df_final['State'])

In [217]:
columns_no_target=['Account Length', "Int'l Plan", 'VMail Plan', 'VMail Message', 'Day Mins',
                   'Day Calls', 'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
                   'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge', 'CustServ Calls']
df_test.to_csv(f's3://{bucket}/{prefix}/churn_data/churn_test.csv')
df_test[columns_no_target].to_csv(f's3://{bucket}/{prefix}/churn_data/churn_test_no_target.csv', index=False)

In [219]:
from sagemaker.amazon.amazon_estimator import image_uris
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from botocore.exceptions import ClientError
import time
from time import gmtime, strftime

image = image_uris.retrieve(region=region, framework='xgboost', version='1.3-1')
train_instance_type = 'ml.m5.xlarge'
train_instance_count = 1
s3_output = f's3://{bucket}/{prefix}/churn_data/training'

experiment_name = 'churn-prediction'

try:
    experiment = Experiment.create(
        experiment_name=experiment_name, 
        description='Training churn prediction models based on telco churn dataset.')
except ClientError as e:
    print(f'{experiment_name} experiment already exists! Reusing the existing experiment.')
    

def launch_training_job(state, train_data_s3, val_data_s3):
    exp_datetime = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
    jobname = f'churn-xgb-{state}-{exp_datetime}'

    # Creating a new trial for the experiment
    exp_trial = Trial.create(experiment_name=experiment_name, 
                             trial_name=jobname)

    experiment_config={
        'ExperimentName': experiment_name,
        'TrialName': exp_trial.trial_name,
        'TrialComponentDisplayName': 'Training'}

    xgb = sagemaker.estimator.Estimator(image,
                                        role,
                                        instance_count=train_instance_count,
                                        instance_type=train_instance_type,
                                        output_path=s3_output,
                                        enable_sagemaker_metrics=True,
                                        sagemaker_session=sess)
    xgb.set_hyperparameters(objective='binary:logistic', num_round=20)
    
    train_input = sagemaker.inputs.TrainingInput(s3_data=train_data_s3, 
                                                 distribution='FullyReplicated', 
                                                 content_type='csv')
    val_input = sagemaker.inputs.TrainingInput(s3_data=val_data_s3, 
                                               distribution='FullyReplicated', 
                                               content_type='csv')
    data_channels={'train': train_input, 'validation': val_input}
    
    xgb.fit(inputs=data_channels, job_name=jobname, 
            experiment_config=experiment_config, wait=False)

    return xgb

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [222]:
dict_estimator = {}
for state in df_final.State.unique()[:5]:
    print(state)
    out_train_csv_s3 = f's3://{bucket}/{prefix}/churn_data/by_state/churn_{state}_train.csv'
    out_val_csv_s3 = f's3://{bucket}/{prefix}/churn_data/by_state/churn_{state}_val.csv'
    df_state = df_train[df_train['State']==state].drop(labels='State', axis=1)
    df_state_train, df_state_val = train_test_split(df_state, test_size=0.1, random_state=42, 
                                                    shuffle=True, stratify=df_state['Churn?'])
#     print(df_state_train.head())
#     print(dict_df_byState[state].shape)
    df_state_train.to_csv(out_train_csv_s3, index=False)
    df_state_val.to_csv(out_val_csv_s3, index=False)
    dict_estimator[state] = launch_training_job(state, out_train_csv_s3, out_val_csv_s3)
    time.sleep(2)
    

PA
            Churn?  Account Length  Int'l Plan  VMail Plan  VMail Message  \
CustomerID                                                                  
401              1              57           0           1            100   
4956             0             187           0           0              0   
3495             0              53           0           0              0   
3075             0              88           1           1            500   
13               1             150           1           0              0   

             Day Mins  Day Calls  Day Charge  Eve Mins  Eve Calls  Eve Charge  \
CustomerID                                                                      
401         11.691693          3    9.197880  7.738775          3    8.060361   
4956         7.681022          4    8.134130  6.855681          9    4.415886   
3495         0.400337          3    4.776804  7.885221          5    0.716801   
3075         3.530491          3    3.817296  5.2277

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: churn-xgb-PA-2021-10-04-20-33-55


SC
            Churn?  Account Length  Int'l Plan  VMail Plan  VMail Message  \
CustomerID                                                                  
2451             1              10           0           1            700   
268              1              82           1           1            700   
4328             1             100           0           0              0   
4705             1               8           0           0              0   
209              1              68           1           1            200   

             Day Mins  Day Calls  Day Charge  Eve Mins  Eve Calls  Eve Charge  \
CustomerID                                                                      
2451         1.868743          6    1.862988  4.299654          2    1.710547   
268          3.421646          5    2.867176  6.763163          1    3.341017   
4328         5.686216          3    4.173494  6.465437          2    4.876660   
4705        12.794809          5    9.377890  2.7238

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: churn-xgb-SC-2021-10-04-20-33-58


MO
            Churn?  Account Length  Int'l Plan  VMail Plan  VMail Message  \
CustomerID                                                                  
1000             0              27           0           0              0   
4537             1             153           1           0              0   
4259             1             136           0           1            400   
512              1              56           1           0              0   
4163             1             128           1           0              0   

            Day Mins  Day Calls  Day Charge  Eve Mins  Eve Calls  Eve Charge  \
CustomerID                                                                     
1000        3.895470          1    4.564941  1.920832          2    4.187244   
4537        8.189338          2    7.174453  6.801406          1    5.320173   
4259        6.152209          4    6.224184  3.424067          7    5.708567   
512         3.971236          0    5.600528  6.532679    

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: churn-xgb-MO-2021-10-04-20-34-01


WY
            Churn?  Account Length  Int'l Plan  VMail Plan  VMail Message  \
CustomerID                                                                  
4680             0             132           1           1              0   
3762             1              16           0           0              0   
4927             1               8           0           1            400   
1853             0              82           1           0              0   
534              1              70           1           1            300   

             Day Mins  Day Calls  Day Charge  Eve Mins  Eve Calls  Eve Charge  \
CustomerID                                                                      
4680         1.187684          4    6.732462  4.819379         12    1.076178   
3762        12.479811          2   10.294631  3.737059          4    7.570004   
4927         6.245781          2    5.118775  8.082050          1    6.763096   
1853         6.609295          4    6.027843  6.3821

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: churn-xgb-WY-2021-10-04-20-34-04


VA
            Churn?  Account Length  Int'l Plan  VMail Plan  VMail Message  \
CustomerID                                                                  
1146             1             139           0           1              0   
4596             0             106           1           0              0   
1761             0              90           0           0              0   
3291             0             110           1           1            400   
4489             1             146           0           1            200   

             Day Mins  Day Calls  Day Charge  Eve Mins  Eve Calls  Eve Charge  \
CustomerID                                                                      
1146         7.380514          3    6.021671  4.735200          2    6.176831   
4596         0.747906          4    2.014188  3.841759          4    3.090763   
1761         5.693790          5    0.131273  3.104597          5    0.740461   
3291         0.540098          4    0.371103  0.3022

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: churn-xgb-VA-2021-10-04-20-34-08


In [223]:
def wait_for_training_job_to_complete(estimator):
    job = estimator.latest_training_job.job_name
    print(f"Waiting for job: {job}")
    status = estimator.latest_training_job.describe()["TrainingJobStatus"]
    while status == "InProgress":
        time.sleep(45)
        status = estimator.latest_training_job.describe()["TrainingJobStatus"]
        if status == "InProgress":
            print(f"{job} job status: {status}")
    print(f"DONE. Status for {job} is {status}\n")

In [224]:
for est in list(dict_estimator.values()):
    wait_for_training_job_to_complete(est)

Waiting for job: churn-xgb-PA-2021-10-04-20-33-55
churn-xgb-PA-2021-10-04-20-33-55 job status: InProgress
churn-xgb-PA-2021-10-04-20-33-55 job status: InProgress
churn-xgb-PA-2021-10-04-20-33-55 job status: InProgress
churn-xgb-PA-2021-10-04-20-33-55 job status: InProgress
DONE. Status for churn-xgb-PA-2021-10-04-20-33-55 is Completed

Waiting for job: churn-xgb-SC-2021-10-04-20-33-58
DONE. Status for churn-xgb-SC-2021-10-04-20-33-58 is Completed

Waiting for job: churn-xgb-MO-2021-10-04-20-34-01
DONE. Status for churn-xgb-MO-2021-10-04-20-34-01 is Completed

Waiting for job: churn-xgb-WY-2021-10-04-20-34-04
DONE. Status for churn-xgb-WY-2021-10-04-20-34-04 is Completed

Waiting for job: churn-xgb-VA-2021-10-04-20-34-08
DONE. Status for churn-xgb-VA-2021-10-04-20-34-08 is Completed



In [227]:
from sagemaker.multidatamodel import MultiDataModel
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

model_PA = dict_estimator['PA'].create_model(role=role, image_uri=image)
model_data_prefix = f's3://{bucket}/{prefix}/churn_data/multi_model_artifacts/'
exp_datetime = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
model_name = f'churn-xgb-mme-{exp_datetime}'
endpoint_name = model_name
hosting_instance_type = 'ml.c5.xlarge'
hosting_instance_count = 1

mme = MultiDataModel(name=model_name,
                     model_data_prefix=model_data_prefix,
                     model=model_PA,  # passing our model - passes container image needed for the endpoint
                     sagemaker_session=sess)
predictor = mme.deploy(initial_instance_count=hosting_instance_count, 
                       instance_type=hosting_instance_type, 
                       endpoint_name=endpoint_name)

predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()

INFO:sagemaker:Creating model with name: churn-xgb-mme-2021-10-04-20-54-34
INFO:sagemaker:Creating endpoint with name churn-xgb-mme-2021-10-04-20-54-34


--------------!

In [228]:
list(mme.list_models())

[]

In [229]:
for state, est in dict_estimator.items():
    artifact_path = est.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts']
    model_name = f'{state}.tar.gz'
    print(model_name)
    # This is copying over the model artifact to the S3 location for the MME.
    mme.add_model(model_data_source=artifact_path, model_data_path=model_name)

PA.tar.gz
SC.tar.gz
MO.tar.gz
WY.tar.gz
VA.tar.gz


In [231]:
def sample_test_data(state):
    sample = df_test[df_test['State']==state].sample(1)
    sample[["Int'l Plan", 'VMail Plan']]=sample[["Int'l Plan", 'VMail Plan']].astype(int)
#     print(sample.values.tolist())
#     print(sample[columns_no_target].values.tolist())
    target = sample['Churn?'].values[0].tolist()
#     sample = sample[columns_no_target].values.tolist()
    sample = sample.values[0][2:].tolist()
    

    return sample, target

In [246]:
start_time = time.time()

state='PA'
test_data=sample_test_data(state)
print(test_data[0])
prediction = predictor.predict(data=test_data[0], 
                               target_model=f'{state}.tar.gz')

duration = time.time() - start_time
print(f'{prediction} vs ground truth {test_data[1]}')
print('It took {:,d} ms\n'.format(int(duration * 1000)))

[3, 1, 0, 0, 2.9633228667054743, 7, 0.05431246149922053, 1.953425131407294, 3, 3.4822767032129565, 0.7477214439020248, 300, 5.508278675828586, 4.8254526221771465, 3, 0.14046832333341452, 7]
[0.03299383446574211] vs ground truth 0
It took 26 ms



In [247]:
start_time = time.time()

state='SC'
test_data=sample_test_data(state)
print(test_data[0])
prediction = predictor.predict(data=test_data[0], 
                               target_model=f'{state}.tar.gz')

duration = time.time() - start_time
print(f'{prediction} vs ground truth {test_data[1]}')
print('It took {:,d} ms\n'.format(int(duration * 1000)))

[169, 0, 1, 400, 4.724202415033885, 3, 5.462342562647127, 7.105265297330521, 1, 3.622453024907145, 2.0820331941213266, 300, 6.456455144540776, 4.509004263306605, 7, 1.5923336262333845, 8]
[0.9330484867095947] vs ground truth 1
It took 1,012 ms



In [248]:
start_time = time.time()

state='VA'
test_data=sample_test_data(state)
print(test_data[0])
prediction = predictor.predict(data=test_data[0], 
                               target_model=f'{state}.tar.gz')

duration = time.time() - start_time
print(f'{prediction} vs ground truth {test_data[1]}')
print('It took {:,d} ms\n'.format(int(duration * 1000)))

[102, 0, 1, 300, 1.3363303518442242, 6, 0.5212972233721498, 4.115880174748177, 0, 3.7230289207986313, 0.9384902279193464, 250, 4.338407257945279, 4.507728017054305, 5, 2.4283743687031314, 6]
[0.04292813315987587] vs ground truth 0
It took 882 ms



In [251]:
list(mme.list_models())

['MO.tar.gz', 'PA.tar.gz', 'SC.tar.gz', 'VA.tar.gz', 'WY.tar.gz']

In [225]:
# predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: churn-xgb-mme-2021-10-04-19-09-27
INFO:sagemaker:Deleting endpoint with name: churn-xgb-mme-2021-10-04-19-09-27


In [218]:
# To remove all experiments from the registry.
# experiment.delete_all(action='--force')